# Jakobi 11

In [1]:
# from classes import *

# Line_13 = Line(bus_p="1", bus_q="3", impedance=complex(0.08, 0.24), half_line_charging_admittance=complex(0.0, 0.025))
# Line_12 = Line(bus_p="1", bus_q="2", impedance=complex(0.02, 0.06), half_line_charging_admittance=complex(0.0, 0.03))

# display(Line_13.build_Line_pq())
# display(Line_12.build_Line_pq())

# bus1 = Bus(bus_id=1, voltage_magnitude=20, voltage_angle=0)
# bus2 = Bus(bus_id=2, voltage_magnitude=30, voltage_angle=5)
# print(bus1)
# print(bus2)

In [2]:
from classes import *
import pandas as pd
import os

pathToFile = os.path.abspath(os.curdir)
df_line_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Line_data")
df_bus_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Bus_data")
line_data = df_line_data.to_dict(orient='records')
bus_data = df_bus_data.to_dict(orient='records')
line_admittences = []
num_buses = len(bus_data)

for line_dict in line_data:
    line = Line(
        bus_p=line_dict['From line'],
        bus_q=line_dict['To line'],
        impedance=complex(line_dict['R[pu]'], line_dict['X[pu]']),
        half_line_charging_admittance=complex(line_dict['Half Line Charging Admittance'])
    )
    line_admittences.append(line.build_Line_Y_bus())

# Initialize YBUS for a N-Bus System
Ybus_init = Y_Bus(line_adm=line_admittences)
YBus = Ybus_init.build_YBUS(num_buses=num_buses)
display('YBus', YBus)


# display('bus_data', bus_data)

'YBus'

,1,2,3,4,5
1,6.305-18.750j,-5.000000+15.000000j,-1.250000+ 3.750000j,0.000000+ 0.000000j,0.00+ 0.00j
2,-5.000+15.000j,10.918333-32.500000j,-1.666667+ 5.000000j,-1.666667+ 5.000000j,-2.50+ 7.50j
3,-1.250+ 3.750j,-1.666667+ 5.000000j,12.971667-38.750000j,-10.000000+30.000000j,0.00+ 0.00j
4,0.000+ 0.000j,-1.666667+ 5.000000j,-10.000000+30.000000j,12.971667-38.750000j,-1.25+ 3.75j
5,0.000+ 0.000j,-2.500000+ 7.500000j,0.000000+ 0.000000j,-1.250000+ 3.750000j,3.79-11.25j


In [ ]:
import math
import numpy as np

##############3 Dette er midlertidig. Får fra Andreas.###################################################
list_dict_power = [{'P1': False, 'P2': True, 'P3': True, 'P4': True, 'P5': True},
                  {'Q1': False, 'Q2': False, 'Q3': True, 'Q4': True, 'Q5': True},
                  {'dirac1': False, 'dirac2': True, 'dirac3': True, 'dirac4': True, 'dirac5': True}]
#########################################################################################################



def J11_calc(bus_data, YBus, list_dict_power): # Mixed form

    row_count = 0
    col_count = 0

    for element in list_dict_power:
        for key, value in element.items():                          # Teller P'er og dirac'er for å vite størrelsen på J11
            if key.startswith('P') and value is True:
                row_count += 1               
            elif key.startswith('d') and value is True:
                col_count += 1           
    df_J11 = pd.DataFrame(0, index=range(1, row_count+1), columns=range(1, col_count+1), dtype=complex) # Lager en tom dataframe med complexe verdier lik 0


# Iterate through each row and column of the J11 array

    for i in range(1, row_count+1):
        for j in range(1, col_count+1):
            if i == j:
                continue
            
            # Tilordner variabler
            vi = abs(bus_data[i]['Assumed bus voltage (pu)'])
            vj = abs(bus_data[j]['Assumed bus voltage (pu)'])
            gij = YBus.at[i, j].real
            sin_ij = math.sin(bus_data[i]['Angle'] - bus_data[j]['Angle'])
            bij = YBus.at[i, j].imag
            cos_ij = math.cos(bus_data[i]['Angle'] - bus_data[j]['Angle'])

            # Beregner elementet i matrisen
            df_J11.at[i, j] = vi * vj *(gij *sin_ij - bij * sin_ij)

    return df_J11
            
test = J11_calc(bus_data=bus_data,
         YBus=YBus,
         list_dict_power=list_dict_power)









    # dPi_dDj = Vi_mag * Vj_mag * (y_line_ij.real * math.sin(Vi_ang-Vj_ang) - y_line_ij.imag * math.cos(Vi_ang-Vj_ang))


    # if (bus_i == bus_j):     # Diagonal
    #     # Lager variabler inni funksjonen
  
    # else:                   # Off-diagonal
    #     Vi_mag = bus_i.voltage_magnitude
    #     Vi_ang = bus_i.voltage_angle
    #     Vj_mag = bus_j.voltage_magnitude
    #     Vj_ang = bus_j.voltage_angle

    #     y_line_ij = 1/line_ij.impedance

    #     dPi_dDj = Vi_mag * Vj_mag * (y_line_ij.real * math.sin(Vi_ang-Vj_ang) - y_line_ij.imag * math.cos(Vi_ang-Vj_ang))
    #     return dPi_dDj



    # # Loop through all pairs of nodes
    # for node_from in range(1, true_count + 1):
    #     for node_to in range(1, true_count + 1):
    #         if node_from == node_to:
    #             continue  # Skip self-connections

    #         connection_found = False

    #         for line_dict in line_data:
    #             if (line_dict['From line'] == node_from and line_dict['To line'] == node_to):
                   

    #                 connection_found = True
    #                 r_value = line_dict['R[pu]']
    #                 x_value = line_dict['X[pu]']
    #                 # Perform your action with r_value and x_value
    #                 print(f"Connection between {node_from} and {node_to}")
    #                 # Exit the loop since you found a connection
    #                 break

    #         if not connection_found:
    #             # No connection between node_from and node_to is found
    #             # Continue with other actions if needed
    #             pass

J11_calc(bus_data, line_data, list_dict_power)

          1         2         3         4
1  0.0+0.0j -0.0+0.0j -0.0+0.0j  0.0+0.0j
2 -0.0+0.0j  0.0+0.0j -0.0+0.0j -0.0+0.0j
3 -0.0+0.0j -0.0+0.0j  0.0+0.0j -0.0+0.0j
4  0.0+0.0j -0.0+0.0j -0.0+0.0j  0.0+0.0j


In [ ]:
# bus_num = 5

# def J11_calc(bus_data, line_data, bus_num):

#     J11 = np.zeros((bus_num, bus_num))

#     # Loop through all pairs of nodes
#     for node_from in range(1, bus_num + 1):
#         for node_to in range(1, bus_num + 1):
#             if node_from == node_to:
#                 continue  # Skip self-connections

#             connection_found = False

#             for line_dict in line_data:
#                 if (line_dict['From line'] == node_from and line_dict['To line'] == node_to):
                   

#                     connection_found = True
#                     r_value = line_dict['R[pu]']
#                     x_value = line_dict['X[pu]']
#                     # Perform your action with r_value and x_value
#                     print(f"Connection between {node_from} and {node_to}")
#                     # Exit the loop since you found a connection
#                     break

#             if not connection_found:
#                 # No connection between node_from and node_to is found
#                 # Continue with other actions if needed
#                 pass

# J11_calc(bus_data, line_data, bus_num)

In [ ]:








    dPi_dDj = Vi_mag * Vj_mag * (y_line_ij.real * math.sin(Vi_ang-Vj_ang) - y_line_ij.imag * math.cos(Vi_ang-Vj_ang))


    from_line = 1
    to_line = 3

    desired_R_value = None      # Zero value kun for å initialisere

    for line_dict in line_data:
        if line_dict['From line'] == from_line and line_dict['To line'] == to_line:
            desired_R_value = line_dict['R[pu]']
            break

    if desired_R_value is not None:
        print(f"The 'R[pu]' value for Line 1 to 3 is: {desired_R_value}")
    else:
        print("Line not found in line_data.")


    return J11

print(J11_calc(bus_data, line_data, P_known))


The 'R[pu]' value for Line 1 to 3 is: 0.08
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
import math




def J11_calc(buses: List[classes.Bus], lines: List[classes.Line]):





   for dictionary in line_data:
        for key, value in dictionary.items():
            if key.startswith('Z12'):
                print(value)





        Vi_mag = bus_i.voltage_magnitude
        Vi_ang = bus_i.voltage_angle
        Vj_mag = bus_j.voltage_magnitude
        Vj_ang = bus_j.voltage_angle



    if (bus_i == bus_j):     # Diagonal
        # Lager variabler inni funksjonen
  
    else:                   # Off-diagonal
        Vi_mag = bus_i.voltage_magnitude
        Vi_ang = bus_i.voltage_angle
        Vj_mag = bus_j.voltage_magnitude
        Vj_ang = bus_j.voltage_angle

        y_line_ij = 1/line_ij.impedance

        dPi_dDj = Vi_mag * Vj_mag * (y_line_ij.real * math.sin(Vi_ang-Vj_ang) - y_line_ij.imag * math.cos(Vi_ang-Vj_ang))
        return dPi_dDj



In [ ]:


def J11_off_diag_calc(bus_i, bus_j, line_ij):    # MIXED FORM: Voltage in polar, impedance in cartesian
    
    if (bus_i == bus_j):     # Diagonal
        # Lager variabler inni funksjonen
        Vi_mag = bus_i.voltage_magnitude
        Vi_ang = bus_i.voltage_angle
        Vj_mag = bus_j.voltage_magnitude
        Vj_ang = bus_j.voltage_angle

        y_line_ij = 1/line_ij.impedance

        dPi_dDj = Vi_mag * Vj_mag * (y_line_ij.real * math.sin(Vi_ang-Vj_ang) - y_line_ij.imag * math.cos(Vi_ang-Vj_ang))
        return dPi_dDj
    else:                   # Off-diagonal


# Bruker grader! Dobbelsjekk at det er rett.
# Plasser i tabell/ liste/ dataframe?
J11_off_diag_calc(bus1, bus2, Line_12)


######################################################################################


# Sett inn liste med spenning og vinkel på alle busser
# Sett inn liste med impedanser på alle linjer

def J11_diag_calc(buses: List[classes.Bus], lines: List[classes.Line]):    # MIXED FORM: Voltage in polar, impedance in cartesian







    num_busses = size.buses

    List[classes.Bus], buses

    # Bruker grader! Sjekk at det er rett.
        for i in range(1, num_busses + 1):
            if i == int(bus_i.bus_id):
                continue
            else:
                dPi_dDi = 2 #Må få laget en liste over spenninger og vinkler i alle busser før formel kan legges inn her.


        return dPi_dDi

    # Plasser i tabell/ liste/ dataframe?
    J11_diag_calc(bus1, bus2, Line_12)

IndentationError: expected an indented block (2590140238.py, line 19)